<a href="https://colab.research.google.com/github/Magesh-Pragmath/Text-Classification-Demo/blob/master/Training_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [ ]:
from transformers import pipeline
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset, DatasetDict

In [ ]:
classifier = pipeline("text-classification", model='distilbert-base-uncased-finetuned-sst-2-english')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


In [ ]:
nlp=classifier("I've been waiting for a HuggingFace course my whole life.")
print(nlp)

[{'label': 'POSITIVE', 'score': 0.9598049521446228}]


In [ ]:
result=classifier("I relly enjoyed studying NLP")
print(result)

[{'label': 'POSITIVE', 'score': 0.9974174499511719}]


In [ ]:
from datasets import load_dataset

ddict = load_dataset("Sp1786/multiclass-sentiment-analysis-dataset")

In [ ]:
df = ddict['train'].to_pandas()

In [ ]:
ddict

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 31232
    })
    validation: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5205
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'sentiment'],
        num_rows: 5206
    })
})

In [ ]:
df.head()

,id,text,label,sentiment
0,9536,"Cooking microwave pizzas, yummy",2,positive
1,6135,Any plans of allowing sub tasks to show up in ...,1,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",2,positive
3,14182,naw idk what ur talkin about,1,neutral
4,17840,That sucks to hear. I hate days like that,0,negative


In [ ]:
df.drop(columns=["label"], axis=1, inplace=True)

In [ ]:
df.head()

,id,text,sentiment
0,9536,"Cooking microwave pizzas, yummy",positive
1,6135,Any plans of allowing sub tasks to show up in ...,neutral
2,17697,"I love the humor, I just reworded it. Like sa...",positive
3,14182,naw idk what ur talkin about,neutral
4,17840,That sucks to hear. I hate days like that,negative


In [ ]:
label=LabelEncoder()
df['sentiment'] = label.fit_transform(df['sentiment'])

In [ ]:
df.head()

,id,text,sentiment
0,9536,"Cooking microwave pizzas, yummy",2
1,6135,Any plans of allowing sub tasks to show up in ...,1
2,17697,"I love the humor, I just reworded it. Like sa...",2
3,14182,naw idk what ur talkin about,1
4,17840,That sucks to hear. I hate days like that,0


In [ ]:
train, validate, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)),int(.8*len(df))])

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [ ]:
train = train.reset_index()[['text','sentiment']]
validate = validate.reset_index()[['text','sentiment']]
test = test.reset_index()[['text','sentiment']]

In [ ]:
train.head()

,text,sentiment
0,@mashable I never did thank you for including ...,2
1,Happy Mothers Day!!,2
2,Welcome home! Glad you made it home safe!,2
3,@__Jasper_Hale__ *Laughs* that`s because I lik...,2
4,Getting ready for work... think I`m catching a...,1


In [ ]:
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
teds = Dataset.from_pandas(test)

In [ ]:
dataset = DatasetDict()
dataset["train"] = tds
dataset["validate"] = vds
dataset["test"] = teds

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 18739
    })
    validate: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 6246
    })
    test: Dataset({
        features: ['text', 'sentiment'],
        num_rows: 6247
    })
})

In [ ]:
from transformers import AutoTokenizer

model_cpkt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_cpkt)

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"],padding=True, truncation=True)

In [ ]:
dataset_encode = dataset.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/18739 [00:00<?, ? examples/s]

Map:   0%|          | 0/6246 [00:00<?, ? examples/s]

Map:   0%|          | 0/6247 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModel
import torch

model_cpkt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_cpkt).to(device)

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items()
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [ ]:
dataset_encode.set_format("torch", columns=["input_ids", "attention_mask", "sentiment"])

dataset_hidden = dataset_encode.map(extract_hidden_states, batched=True)  #Not able to process fully

In [ ]:
dataset_hidden = dataset_encode.map(extract_hidden_states, batched=True, batch_size=16)

Map:   0%|          | 0/18739 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
for bs in [1024, 512, 128, 64, 32, 16, 8, 4, 2, 1]:
    try:
        print(f"Trying batch size = {bs}")
        dataset_hidden = dataset_encode.map(extract_hidden_states, batched=True, batch_size=bs)
        print(f"✅ Worked at batch size = {bs}")
        break
    except RuntimeError as e:
        print(f"❌ Failed at batch size = {bs}")
        torch.cuda.empty_cache()


Trying batch size = 1024


Map:   0%|          | 0/18739 [00:00<?, ? examples/s]

In [ ]:
dataset_hidden["train"].column_names

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = (AutoModelForSequenceClassification.from_pretrained(model_cpkt, num_labels=num_labels).to(device))

In [ ]:
from sklearn.metrics import accuracy_score,f1_score

def compute_metrics(pred):
  labels = pred.lable_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(label, preds, average="weighted")
  acc = accuracy_score(labels, preds)
  return {"Accuracy: ",acc , "F1_score:",f1}

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(dataset_encode["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(
    output_dir=model_name,  # where to save model + logs
    num_train_epochs=2,     # how many times to loop over the dataset
    learning_rate=2e-5,     # how fast the model learns (usually between 1e-5 to 5e-5)
    per_device_train_batch_size=batch_size,  # batch size per GPU (or CPU if no GPU)
    per_device_eval_batch_size=batch_size,   # same, but for evaluation
    weight_decay=0.01,      # helps prevent overfitting (L2 regularization)
    evaluation_strategy="epoch",  # run eval at the end of every epoch
    disable_tqdm=False,     # whether to show the progress bar (set to True to hide it)
    logging_steps=logging_steps,  # how often to log metrics
    push_to_hub=False,      # set to True if you want to push your model to Hugging Face Hub
    log_level="error"       # only show error messages in logs
)

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=emotions_encoded["train"],
                  eval_dataset=emotions_encoded["validation"],
                  tokenizer=tokenizer)

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 6362044367749746505
 xla_global_id: -1]